In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

atlas_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/x64/Release"
sys.path.append(atlas_path)


from AtlasPy.core import Hydra, Portfolio, Strategy
from AtlasPy.ast import *

In [2]:
exchange_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/AtlasPy/src/exchangeVBTMulti"
strategy_id = "test_strategy"
exchange_id = "test_exchange"
portfolio_id = "test_portfolio"

hydra = Hydra()
intial_cash = 100.0
exchange = hydra.addExchange(exchange_id, exchange_path, "%Y-%m-%d %H:%M:%S")
portfolio = hydra.addPortfolio(portfolio_id, exchange, intial_cash)
hydra.build()

In [3]:
st = time.time()
fast_n = 6
slow_n = 12
windows = np.arange(50, 200)

close = AssetReadNode.make("Close", 0, exchange)
fast_ma = MeanObserverNode(
    close,
    fast_n
)
slow_ma = MeanObserverNode(
    close,
    slow_n
)

spread = AssetOpNode.make(fast_ma, slow_ma, AssetOpType.SUBTRACT)
spread_filter = ExchangeViewFilter(ExchangeViewFilterType.GREATER_THAN, 0.0, None)
exchange_view = ExchangeViewNode.make(exchange, spread, spread_filter)

fast_ma_dim = GridDimensionObserver.make(
    name="Fast MA",
    dimension_values=windows,
    observer_base=fast_ma,
    observer_child=spread,
    swap_addr=spread.getSwapLeft()
)
slow_ma_dim = GridDimensionObserver.make(
    name="Slow MA",
    dimension_values=windows,
    observer_base=slow_ma,
    observer_child=spread,
    swap_addr=spread.getSwapRight()
)

allocation = AllocationNode.make(
    exchange_view,
    AllocationType.UNIFORM,
    None,
    0
)
strategy_node = StrategyNode.make(allocation, portfolio)
strategy = hydra.addStrategy(Strategy(strategy_id, strategy_node, 1.0), True)
grid = strategy.setGridDimmensions((fast_ma_dim, slow_ma_dim), GridType.UPPER_TRIANGULAR)
grid.enableTracerHistory(TracerType.NLV)
strategy.enableTracerHistory(TracerType.NLV)
hydra.run()

nlv = strategy.getHistory(TracerType.NLV)
returns = nlv[-1] / nlv[0] - 1

et = time.time()
print(f'Elapsed time: {1000*(et-st):.2f} ms')
print(f'Total return: {returns}')

Elapsed time: 7094.48 ms
Total return: -1.0000268177453029


In [4]:
nlv = strategy.getHistory(TracerType.NLV)
timestamps = pd.to_datetime(exchange.getTimestamps())
returns = nlv[-1] / nlv[0] - 1
print("Returns: ", returns)

Returns:  -1.0000268177453029


In [5]:
rows = grid.rows()
cols = grid.cols()

pairs = []
nlvs = []
returns_dict = {}
for i in range(rows):
    for j in range(cols):
        if i >= j:
            continue

        pairs.append((windows[i], windows[j]))
        tracer = grid.getTracer(i, j)
        nlv = tracer.getHistory(TracerType.NLV)
        returns = nlv[-1] / nlv[0] - 1
        returns_dict[pairs[-1]] = returns
        nlvs.append(nlv)
        
avg = np.mean(list(returns_dict.values()))
print(f'Average return: {avg}')

Average return: -0.9949167435996242


In [6]:
returns_dict

{(50, 51): -1.0001008236978888,
 (50, 52): -1.0001112077928278,
 (50, 53): -1.000115443517421,
 (50, 54): -1.0001191889156453,
 (50, 55): -1.0001202115747372,
 (50, 56): -0.9924330393863247,
 (50, 57): -0.9919155081485467,
 (50, 58): -0.9922955169051937,
 (50, 59): -0.9927040765024415,
 (50, 60): -1.0000164161233358,
 (50, 61): -1.0000179798671343,
 (50, 62): -1.0000168658135533,
 (50, 63): -1.0000171848803137,
 (50, 64): -1.0000175922979715,
 (50, 65): -0.9985006044938318,
 (50, 66): -1.0000183561839031,
 (50, 67): -1.0000180510758017,
 (50, 68): -1.0000186593532832,
 (50, 69): -1.0000195562971426,
 (50, 70): -0.9983048561909342,
 (50, 71): -1.0000135022710017,
 (50, 72): -0.9949851307974958,
 (50, 73): -0.9944668986538558,
 (50, 74): -0.99446091510698,
 (50, 75): -0.99446091510698,
 (50, 76): -1.0000163034834335,
 (50, 77): -0.9935241640658036,
 (50, 78): -0.9933661369704572,
 (50, 79): -0.9934251091112245,
 (50, 80): -0.9933128466411425,
 (50, 81): -0.9931985705505435,
 (50, 82): -0